In [1]:
# Импорт библиотек
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report, accuracy_score
from scipy.stats import randint, uniform
import xgboost as xgb
from pathlib import Path
import time

# Загрузка данных
X_normalized = pd.read_parquet('../data/processed/X_normalized.parquet')
y_multiclass = pd.read_parquet('../data/processed/y_multiclass.parquet')['attack_cat']

# Загрузка label encoder
multiclass_package = joblib.load('../models/best_multiclass_classifier.joblib')
label_encoder = multiclass_package['label_encoder']

# Кодируем метки
y_encoded = label_encoder.transform(y_multiclass)

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Данные для настройки:")
print(f"Тренировочные: {X_train.shape}")
print(f"Тестовые: {X_test.shape}")

Данные для настройки:
Тренировочные: (206138, 49)
Тестовые: (51535, 49)


In [2]:
# Пространство гиперпараметров для RandomizedSearch
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),  # от 0.6 до 1.0
    'colsample_bytree': uniform(0.6, 0.4),
    'reg_alpha': uniform(0, 2),
    'reg_lambda': uniform(0, 2),
    'gamma': uniform(0, 1)
}

print("Пространства поиска гиперпараметров:")
for param, distribution in param_dist.items():
    print(f"  {param}: {distribution}")

Пространства поиска гиперпараметров:
  n_estimators: <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000021EF5EABA10>
  max_depth: <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000021EFC45E350>
  learning_rate: <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000021EF5EAB770>
  subsample: <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000021EFC45DF90>
  colsample_bytree: <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000021EFC45E0D0>
  reg_alpha: <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000021EFC3CB360>
  reg_lambda: <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000021EFC3CB5C0>
  gamma: <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000021EFC6B1010>


In [3]:
# Создаем базовую модель
base_model = xgb.XGBClassifier(
    random_state=42,
    eval_metric='mlogloss',
    use_label_encoder=False
)

# RandomizedSearchCV
print("\nПоиск лучших гиперпараметров...")
start_time = time.time()

random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=50,  # Количество комбинаций для проверки
    cv=3,       # 3-fold кросс-валидация
    scoring='accuracy',
    random_state=42,
    n_jobs=-1,  # Использовать все ядра процессора
    verbose=1
)

# Запускаем поиск
random_search.fit(X_train, y_train)

end_time = time.time()
print(f"Поиск завершен за {(end_time - start_time)/60:.1f} минут")


Поиск лучших гиперпараметров...
Fitting 3 folds for each of 50 candidates, totalling 150 fits


C:\Users\ro517\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [18:31:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Поиск завершен за 26.7 минут


In [4]:
# Анализ лучших параметров
print("\nРезультаты настройки:")

print("Лучшие параметры:")
best_params = random_search.best_params_
for param, value in best_params.items():
    print(f"  {param}: {value}")

print(f"\nЛучшая точность (кросс-валидация): {random_search.best_score_:.4f}")

# Сравнение с исходной моделью
original_model = multiclass_package['model']
original_accuracy = accuracy_score(y_test, original_model.predict(X_test))
tuned_accuracy = accuracy_score(y_test, random_search.best_estimator_.predict(X_test))

print(f"\nсравнение с исходной:")
print(f"Исходная модель: {original_accuracy:.4f}")
print(f"Настроенная модель: {tuned_accuracy:.4f}")
print(f"Улучшение: {tuned_accuracy - original_accuracy:.4f}")

if tuned_accuracy > original_accuracy:
    print("Настройка дала улучшение!")
else:
    print("Улучшения нет, но модель может быть более стабильной")


Результаты настройки:
Лучшие параметры:
  colsample_bytree: 0.6557975442608167
  gamma: 0.29214464853521815
  learning_rate: 0.1199085529881075
  max_depth: 8
  n_estimators: 274
  reg_alpha: 1.2367720186661746
  reg_lambda: 0.7649239825343255
  subsample: 0.9932923543227152

Лучшая точность (кросс-валидация): 0.8234

сравнение с исходной:
Исходная модель: 0.8200
Настроенная модель: 0.8245
Улучшение: 0.0045
Настройка дала улучшение!


In [5]:
# Детальная оценка улучшенной модели
print("\nДетальная оценка улучшенной модели")

y_pred_tuned = random_search.best_estimator_.predict(X_test)
y_pred_original = original_model.predict(X_test)

from sklearn.metrics import classification_report

print("Отчет по классификации (улучшенная модель):")
print(classification_report(
    label_encoder.inverse_transform(y_test),
    label_encoder.inverse_transform(y_pred_tuned)
))

# Сравнение производительности по классам
def compare_class_performance(y_true, y_pred_orig, y_pred_tuned, label_encoder):
    classes = label_encoder.classes_
    comparison = []
    
    for class_name in classes:
        class_mask = y_true == label_encoder.transform([class_name])[0]
        
        if class_mask.sum() > 0:  # если есть примеры этого класса
            acc_orig = accuracy_score(y_true[class_mask], y_pred_orig[class_mask])
            acc_tuned = accuracy_score(y_true[class_mask], y_pred_tuned[class_mask])
            
            comparison.append({
                'Class': class_name,
                'Original_Accuracy': acc_orig,
                'Tuned_Accuracy': acc_tuned,
                'Improvement': acc_tuned - acc_orig
            })
    
    comparison_df = pd.DataFrame(comparison)
    return comparison_df.sort_values('Improvement', ascending=False)

# Запускаем сравнение
comparison_df = compare_class_performance(y_test, y_pred_original, y_pred_tuned, label_encoder)

print("\nУлучшение по классам:")
print(comparison_df.round(4))


Детальная оценка улучшенной модели
Отчет по классификации (улучшенная модель):
                precision    recall  f1-score   support

      Analysis       0.87      0.11      0.20       535
      Backdoor       0.96      0.10      0.18       466
           DoS       0.67      0.06      0.11      3271
      Exploits       0.61      0.93      0.74      8905
       Fuzzers       0.68      0.54      0.60      4849
       Generic       0.99      0.98      0.99     11774
        Normal       0.89      0.94      0.92     18600
Reconnaissance       0.92      0.77      0.84      2798
     Shellcode       0.52      0.61      0.56       302
         Worms       0.39      0.31      0.35        35

      accuracy                           0.82     51535
     macro avg       0.75      0.54      0.55     51535
  weighted avg       0.83      0.82      0.80     51535


Улучшение по классам:
            Class  Original_Accuracy  Tuned_Accuracy  Improvement
9           Worms             0.2571        

In [6]:
# Сохранение улучшенной модели
print("\nСохранение улучшенной модели")

models_path = Path("../models")
models_path.mkdir(parents=True, exist_ok=True)

# Создаем package с улучшенной моделью и label encoder
tuned_model_package = {
    'model': random_search.best_estimator_,
    'label_encoder': label_encoder,
    'feature_names': X_normalized.columns.tolist(),
    'best_params': best_params,
    'tuning_results': {
        'original_accuracy': original_accuracy,
        'tuned_accuracy': tuned_accuracy,
        'improvement': tuned_accuracy - original_accuracy
    }
}

# Сохраняем
joblib.dump(tuned_model_package, models_path / "tuned_multiclass_classifier.joblib")
print("Улучшенная модель сохранена")

# Сохраняем сравнение
reports_path = Path("../reports")
comparison_df.to_csv(reports_path / "hyperparameter_tuning_comparison.csv", index=False)
print("Результаты сравнения сохранены")


Сохранение улучшенной модели
Улучшенная модель сохранена
Результаты сравнения сохранены


In [7]:
# Генерация финального отчета
print("\nФИНАЛЬНЫЙ ОТЧЕТ ПО НАСТРОЙКЕ")

# Анализ лучших параметров
best_params_analysis = """
АНАЛИЗ ЛУЧШИХ ПАРАМЕТРОВ:

"""
for param, value in best_params.items():
    best_params_analysis += f"- {param}: {value}\n"

improved_classes = comparison_df[comparison_df['Improvement'] > 0]['Class'].tolist()
worsened_classes = comparison_df[comparison_df['Improvement'] < 0]['Class'].tolist()

report = f"""
ОТЧЕТ ПО НАСТРОЙКЕ ГИПЕРПАРАМЕТРОВ
{'='*50}

ОБЩИЕ РЕЗУЛЬТАТЫ:
- Исходная точность: {original_accuracy:.4f}
- Точность после настройки: {tuned_accuracy:.4f}
- Улучшение: {tuned_accuracy - original_accuracy:.4f}

{best_params_analysis}
АНАЛИЗ ПО КЛАССАМ:
- Улучшились: {', '.join(improved_classes) if improved_classes else 'нет'}
- Ухудшились: {', '.join(worsened_classes) if worsened_classes else 'нет'}

ВЫВОДЫ:
{'Настройка успешна - модель улучшена' if tuned_accuracy > original_accuracy else 'Существенного улучшения не достигнуто'}

РЕКОМЕНДАЦИИ:
1. Использовать улучшенную модель для продакшена
2. Продолжить мониторинг производительности
3. Рассмотреть более тщательный поиск для проблемных классов

СОХРАНЕННЫЕ ФАЙЛЫ:
- models/tuned_multiclass_classifier.joblib
- reports/hyperparameter_tuning_comparison.csv
"""

print(report)

# Сохраняем отчет
with open(reports_path / "hyperparameter_tuning_report.txt", "w", encoding='utf-8') as f:
    f.write(report)


ФИНАЛЬНЫЙ ОТЧЕТ ПО НАСТРОЙКЕ

ОТЧЕТ ПО НАСТРОЙКЕ ГИПЕРПАРАМЕТРОВ

ОБЩИЕ РЕЗУЛЬТАТЫ:
- Исходная точность: 0.8200
- Точность после настройки: 0.8245
- Улучшение: 0.0045


АНАЛИЗ ЛУЧШИХ ПАРАМЕТРОВ:

- colsample_bytree: 0.6557975442608167
- gamma: 0.29214464853521815
- learning_rate: 0.1199085529881075
- max_depth: 8
- n_estimators: 274
- reg_alpha: 1.2367720186661746
- reg_lambda: 0.7649239825343255
- subsample: 0.9932923543227152

АНАЛИЗ ПО КЛАССАМ:
- Улучшились: Worms, Fuzzers, Shellcode, Analysis, DoS, Exploits, Backdoor
- Ухудшились: Generic, Reconnaissance, Normal

ВЫВОДЫ:
Настройка успешна - модель улучшена

РЕКОМЕНДАЦИИ:
1. Использовать улучшенную модель для продакшена
2. Продолжить мониторинг производительности
3. Рассмотреть более тщательный поиск для проблемных классов

СОХРАНЕННЫЕ ФАЙЛЫ:
- models/tuned_multiclass_classifier.joblib
- reports/hyperparameter_tuning_comparison.csv

